In [10]:
from PIL import Image 
import pytesseract 
import sys 
from pdf2image import convert_from_path 
import os 
from pymongo import MongoClient
import pandas as pd
import re

In [11]:
client = MongoClient("localhost", 27017)
db = client["Batch_GI"]
coll = "Batch_GI"
db_coll = db[coll]

In [12]:
path = "/home/controllingde/G/Batch_GI/"

In [13]:
files = os.listdir(path)

In [14]:
pdffiles = [path + x for x in files if x.find(".pdf") > 0]

In [35]:
colnames = ['batch', 'item', '_id', 'FS', 'GI', 'MHD', 'filename']
total_df = pd.DataFrame(columns=colnames)

In [36]:
datepattern = ".(1[0-2]|0[1-9]|\d)\/([2-9]\d[1-9]\d|[1-9]\d)."
batchsign = "Ch.-B.:"

for f in pdffiles:
    pages = convert_from_path(f, 500)     
    
    image_counter = 1
    
    for p in pages:
        filename = "page_" + str(image_counter) + ".jpg"
        p.save(filename, "JPEG")
        image_counter += 1
    filelimit = image_counter-1
    
    impdict = {}
    for i in range(1, filelimit + 1):
        filename = "page_"+str(i)+".jpg"
        text = str(((pytesseract.image_to_string(Image.open(filename)))))
        text = text.replace('-\n', '')  
    
        batchind_start = text.rindex(batchsign)
    
        batchsub1 = text[(batchind_start + len(batchsign) + 1) : ]
        impdict["batch"] = batchsub1[0: batchsub1.find(" ")]
        impdict["item"] = batchsub1[(batchsub1.find(" ") + 1): batchsub1.find("\n")]
    
        impdict["_id"] = impdict["batch"] + "_" + impdict["item"]
    
        FSsub = batchsub1[batchsub1.find("FS-"): ]
        impdict["FS"] = FSsub[0: FSsub.find("\n")]

        GIsub = batchsub1[batchsub1.find("GI-"): ]
        impdict["GI"] = GIsub[0: GIsub.find("\n")]

        #FOLsub = batchsub1[batchsub1.find("FOL-"): ]
        #impdict["FOL"] = FOLsub[0: FOLsub.find("\n")]
        
        MHD = re.search(datepattern, text)
        impdict["MHD"] = text[MHD.start() : MHD.end()-1]
             
        impdict["filename"] = f
        impdict["fulltext"] = text.split("\n")
        
    db_coll.remove({"_id": impdict["_id"]})
    db_coll.insert_one(impdict)
    
    del impdict["fulltext"]
    pre_df = pd.DataFrame.from_dict(impdict, orient="index").transpose()
    total_df = pd.concat([total_df, pre_df])
    
total_df.to_csv(path + "test01.csv")
total_df.set_index(["_id"], inplace=True, verify_integrity=True)
total_df.head()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:44: DeprecationWarning: remove is deprecated. Use delete_one or delete_many instead.


,batch,item,FS,GI,MHD,filename
_id,,,,,,
VY0519002-A_1010526,VY0519002-A,1010526,FS-1411-100-1117-02-APL,GI-1411-1117-01-APL,01/2022,/home/controllingde/G/Batch_GI/GItest01.pdf
PZ0418194-BU_1010807,PZ0418194-BU,1010807,"FS-1295-12x4 0,5-0718-01-APL eked",GI-1295-081 7-01-APL,10/2020,/home/controllingde/G/Batch_GI/GItest02.pdf


In [ ]:
#for checkng monodb
r = db_coll.find()
for l in r:
    print(l)
    print("\n")